# Exercise sheet 9
### Due 20/01/2023, 14:00 
- Simon Pick
- Sascha Barz
- Fabian Kampshoff

## A simple particle detector

Consider a fictional particle detector built as a spherical volume of radius $R$. The events (particles) recorded by the detector have the following observables:
- cartesian cordinates of the interaction vertex $(x,y,z)$ referenced to the center of the sphere;
- direction of the particle, represented in the form of direction cosines $(d_x, d_y, d_z)$
- reconstructed energy ($E_{rec}$);
- relative detection time ($t$), for simplicity you can just use a float for this (although you have learned about proper timekeeping in the lecture).

The detector is expected to record background events at a constant rate $r_b$. We want to simulate the detector background according to these (intuitive) properties:
- uniform spatial distribution inside the detector volume;
- uniform direction distribution;
- uniform time distribution;
- the true energy of the background particles ($E_b$) has a normal distribution with given mean $\mu_{E_b}$ and standard deviation $\sigma_{E_b}$;
- due to the limited energy resolution of the detector (that we represent with a parameter $\Delta E_{rec}$), a particle of energy $E_{true}$ will be reconstructed as having energy $E_{rec}$, where $E_{rec}$ follows a normal distribution with mean $E$ and standard deviation $\Delta E_{rec} \cdot E_{true}$.


### 1. The module
Create a python module `detector.py` providing a class `Detector`.

The class must provide a `generate_background()` method that takes as an argument a duration `T` (livetime) and returns a `numpy` array representing a sample of background events with time between `0` and `T`, having the properties described above. The columns of the array should represent the values introduced before:
- `x`, `y`, `z`, `d_x`, `d_y`, `d_z`, `t`, `E_true`, `E_rec`.

To make your code more tidy, group the properties of the background in a separate class, following this example skeleton code:

In [ ]:
### 
### IMPORTANT
### COMMENT OUT OR DELETE THIS CELL ONCE YOU HAVE CREATED YOUR MODULE WITH YOUR CODE IN THERE!
###
import numpy as np

class BackgroundSource:
    def __init__(self, r_b : float, mu_Eb : float, sigma_Eb: float):
        self.r_b = r_b
        self.mu_Eb = mu_Eb
        self.sigma_Eb = sigma_Eb
        pass
    # MORE CODE HERE, IF NECESSARY

class Detector:
    def __init__(self, R : float, delta_Erec: float, background_source : BackgroundSource):
        self.R = R
        self.delta_Erec = delta_Erec
        self.background_source = background_source
        pass

    def generate_background(self, T : int):
        # Generate the number of background events
        num_events = int(self.background_source.r_b * T)
        
        # Generate random x, y, z coordinates within the sphere
        x = (np.random.rand(num_events) - 0.5) * 2 * self.R
        y = (np.random.rand(num_events) - 0.5) * 2 * self.R
        z = (np.random.rand(num_events) - 0.5) * 2 * self.R
        
        # Generate random direction cosines
        d_x = np.random.rand(num_events) - 0.5
        d_y = np.random.rand(num_events) - 0.5
        d_z = np.random.rand(num_events) - 0.5
        
        # Generate random times between 0 and T
        t = np.random.rand(num_events) * T
        
        # Generate true energies with normal distribution
        E_true = np.random.normal(self.background_source.mu_Eb, self.background_source.sigma_Eb, num_events)
        
        # Generate reconstructed energies with normal distribution
        E_rec = np.random.normal(E_true, self.delta_Erec * E_true, num_events)
        
        # Create an array of the events
        events = np.column_stack((x, y, z, d_x, d_y, d_z, t, E_true, E_rec))
        
        return events

    # MORE CODE HERE, IF NECESSARY

In [3]:
# After you have created your module, you should import the required symbols by activating the following line.
from detector import Detector, BackgroundSource

### TEST PARAMETERS ###
radius = 30.0                   # m
energy_resolution = 0.25

background_rate = 10            # s^{-1}
background_energy_mean  = 1    # MeV
background_energy_sigma = 0.05 # MeV

livetime = 86400                # s

### TEST ###
bg = BackgroundSource(r_b = background_rate, mu_Eb = background_energy_mean, sigma_Eb = background_energy_sigma)

det = Detector(R = radius, delta_Erec = energy_resolution, background_source = bg)

background_sample = det.generate_background(livetime)

**Remember**: in most Jupyter installations you have to restart your kernel every time you make a modification to a module in order to run the updated code.

### 2. The script
Provide a script that generates a sample of background events of duration `T` and saves it to a file using the `numpy` builtin functionality. We take this chance to learn the basics of `argparse` to read arguments and options from the command line.

Now take a look at the provided example `generate_events.py` script and modify it to provide the required functionality.

In [4]:
# Try and run this from your terminal, without the starting `!`
python generate_events.py --background_energy 10.0 --output_file "out.txt" 86400
# At the moment, this script does nothing aside from printing.

SyntaxError: invalid syntax (651126210.py, line 2)

Here `86400` is the value of the `livetime` positional argument, while `--background_energy` and `--output_file` are optional arguments. Check how the two are defined differently in `generate_events.py`.

### 3. The benchmark
In Jupyter, load the numpy file produced by your script and produce the following plots:
- one scatter plot $E_{rec}$ vs $E_{true}$;
- one histogram for each of the three spherical coordinates of the interaction vertices: radial distance ($\rho$), zenithal angle ($\theta$) and azimuthal angle ($\phi$). You should calculate these coordinates from $(x, y, z)$. Plot also two additional histograms: one for $\sin(\phi)$ and one for $\cos(\theta)$. Make sure to choose appropriate ranges and number of bins for the histograms, do not leave them as defaults!
- one histogram for each of the three direction cosines $(d_x, d_y, d_z)$;
- one histogram for the distribution of the **time difference between any two consecutive events**. Calculate the mean time difference and mark it on the histogram. (Bonus questions: does this distribution look familiar to you? Does it have some relationship with the input parameters?)

In [7]:
import numpy as np
background_sample = np.loadtxt("out.txt")

import matplotlib.pyplot as plt

E_true = background_sample[:, 7]
E_rec = background_sample[:, 8]
plt.scatter(E_true, E_rec)
plt.xlabel("E_true (MeV)")
plt.ylabel("E_rec (MeV)")
plt.show()

x = background_sample[:, 0]
y = background_sample[:, 1]
z = background_sample[:, 2]

rho = np.sqrt(x**2 + y**2 + z**2)
theta = np.arccos(z / rho)
phi = np.arctan2(y, x)

plt.hist(rho, bins=50, range=(0, args.detector_radius), density=True)
plt.xlabel("Radial distance (m)")
plt.show()

plt.hist(theta, bins=50, density=True)
plt.xlabel("Zenithal angle (rad)")
plt.show()

plt.hist(phi, bins=50, density=True)
plt.xlabel("Azimuthal angle (rad)")
plt.show()

plt.hist(np.sin(phi), bins=50, density=True)
plt.xlabel("sin(phi)")
plt.show()

plt.hist(np.cos(theta), bins=50, density=True)
plt.xlabel("cos(theta)")
plt.show()

d_x = background_sample[:, 3]
d_y = background_sample[:, 4]
d_z = background_sample[:, 5]

plt.hist(d_x, bins=50, density=True)
plt.xlabel("d_x")
plt.show()

plt.hist(d_y, bins=50, density=True)
plt.xlabel("d_y")
plt.show()

plt.hist(d_z, bins=50, density=True)
plt.xlabel("d_z")
plt.show()



OSError: out.txt not found.